In [1]:
import pandas as pd
import numpy as np

In [2]:
weather = pd.read_csv(r"/Users/williamchen/Downloads/IDS/rain/weather.csv",index_col=0)

In [3]:
weather.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,...,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,...,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,...,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,...,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,...,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No


In [4]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 142192
Data columns (total 22 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Date           142193 non-null  object 
 1   Location       142193 non-null  object 
 2   MinTemp        141556 non-null  float64
 3   MaxTemp        141871 non-null  float64
 4   Rainfall       140787 non-null  float64
 5   Evaporation    81350 non-null   float64
 6   Sunshine       74377 non-null   float64
 7   WindGustDir    132863 non-null  object 
 8   WindGustSpeed  132923 non-null  float64
 9   WindDir9am     132180 non-null  object 
 10  WindDir3pm     138415 non-null  object 
 11  WindSpeed9am   140845 non-null  float64
 12  WindSpeed3pm   139563 non-null  float64
 13  Humidity9am    140419 non-null  float64
 14  Humidity3pm    138583 non-null  float64
 15  Pressure9am    128179 non-null  float64
 16  Pressure3pm    128212 non-null  float64
 17  Cloud9am       88536 non-null

In [5]:
#探索缺失值
weather.isnull().mean() #缺失值所占总值的比例 isnull().sum(全部的True)/X.shape[0]
#我们要有不同的缺失值填补策略

Date             0.000000
Location         0.000000
MinTemp          0.004480
MaxTemp          0.002265
Rainfall         0.009888
Evaporation      0.427890
Sunshine         0.476929
WindGustDir      0.065615
WindGustSpeed    0.065193
WindDir9am       0.070418
WindDir3pm       0.026570
WindSpeed9am     0.009480
WindSpeed3pm     0.018496
Humidity9am      0.012476
Humidity3pm      0.025388
Pressure9am      0.098556
Pressure3pm      0.098324
Cloud9am         0.377353
Cloud3pm         0.401525
Temp9am          0.006358
Temp3pm          0.019171
RainTomorrow     0.000000
dtype: float64

In [6]:
# 处理缺失值

In [7]:
#首先找出，分类型特征都有哪些，先不处理Date，不处理Y=RainTomorrow
cate = weather.columns[weather.dtypes == "object"].tolist()
cate

['Date', 'Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainTomorrow']

In [8]:
cate = ['Climate',
 'WindGustDir',
 'WindDir9am',
 'WindDir3pm',
 'RainToday',
 'Cloud9am',
 'Cloud3pm']

In [10]:
#对于分类型特征，我们使用众数来进行填补

from sklearn.impute import SimpleImputer #0.20, conda, pip

si = SimpleImputer(missing_values=np.nan,strategy="most_frequent")
#注意，我们使用训练集数据来训练我们的填补器，本质是在生成训练集中的众数
si.fit(weather.loc[:,cate])
#然后我们用训练集中的众数来同时填补训练集和测试集
weather.loc[:,cate] = si.transform(weather.loc[:,cate])

In [11]:
#查看分类型特征是否依然存在缺失值
weather.loc[:,cate].isnull().mean()

WindGustDir    0.0
WindDir9am     0.0
WindDir3pm     0.0
Cloud9am       0.0
Cloud3pm       0.0
dtype: float64

In [12]:
#探索缺失值
weather.isnull().mean() #缺失值所占总值的比例 isnull().sum(全部的True)/X.shape[0]
#我们要有不同的缺失值填补策略

Date             0.000000
Location         0.000000
MinTemp          0.004480
MaxTemp          0.002265
Rainfall         0.009888
Evaporation      0.427890
Sunshine         0.476929
WindGustDir      0.000000
WindGustSpeed    0.065193
WindDir9am       0.000000
WindDir3pm       0.000000
WindSpeed9am     0.009480
WindSpeed3pm     0.018496
Humidity9am      0.012476
Humidity3pm      0.025388
Pressure9am      0.098556
Pressure3pm      0.098324
Cloud9am         0.000000
Cloud3pm         0.000000
Temp9am          0.006358
Temp3pm          0.019171
RainTomorrow     0.000000
dtype: float64

In [13]:
#将所有的分类型变量编码为数字，一个类别是一个数字
from sklearn.preprocessing import OrdinalEncoder #只允许二维以上的数据进行输入
oe = OrdinalEncoder()

In [14]:
#利用训练集进行fit
oe = oe.fit(weather.loc[:,cate])

In [15]:
#用训练集的编码结果来编码训练和测试特征矩阵
#在这里如果测试特征矩阵报错，就说明测试集中出现了训练集中从未见过的类别
weather.loc[:,cate] = oe.transform(weather.loc[:,cate])

In [16]:
col = weather.columns.tolist()
col

['Date',
 'Location',
 'MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustDir',
 'WindGustSpeed',
 'WindDir9am',
 'WindDir3pm',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Cloud9am',
 'Cloud3pm',
 'Temp9am',
 'Temp3pm',
 'RainTomorrow']

In [17]:
# col 非cate
for i in cate:
    col.remove(i)

In [18]:
col

['Date',
 'Location',
 'MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustSpeed',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Temp9am',
 'Temp3pm',
 'RainTomorrow']

In [19]:
col = ['MinTemp',
 'MaxTemp',
 'Rainfall',
 'Evaporation',
 'Sunshine',
 'WindGustSpeed',
 'WindSpeed9am',
 'WindSpeed3pm',
 'Humidity9am',
 'Humidity3pm',
 'Pressure9am',
 'Pressure3pm',
 'Temp9am',
 'Temp3pm']

In [20]:
#实例化模型，填补策略为"mean"表示均值
impmean = SimpleImputer(missing_values=np.nan,strategy = "mean")
#用训练集来fit模型
impmean = impmean.fit(weather.loc[:,col])
#分别在训练集和测试集上进行均值填补
weather.loc[:,col] = impmean.transform(weather.loc[:,col])

In [21]:
weather.isnull().mean()

Date             0.0
Location         0.0
MinTemp          0.0
MaxTemp          0.0
Rainfall         0.0
Evaporation      0.0
Sunshine         0.0
WindGustDir      0.0
WindGustSpeed    0.0
WindDir9am       0.0
WindDir3pm       0.0
WindSpeed9am     0.0
WindSpeed3pm     0.0
Humidity9am      0.0
Humidity3pm      0.0
Pressure9am      0.0
Pressure3pm      0.0
Cloud9am         0.0
Cloud3pm         0.0
Temp9am          0.0
Temp3pm          0.0
RainTomorrow     0.0
dtype: float64

In [24]:
# 处理日期

In [25]:
weather["Rainfall"].isnull().sum()
#假设你没有下雨
#复制你的空值

0

In [26]:
# 加一个新变量RainToday, 降雨量大于1则yes
weather.loc[weather["Rainfall"] >= 1,"RainToday"] = "Yes"
weather.loc[weather["Rainfall"] < 1,"RainToday"] = "No"
weather.loc[weather["Rainfall"] == np.nan,"RainToday"] = np.nan

In [27]:
weather.head(50)

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,RainToday
0,2008-12-01,Albury,13.4,22.9,0.600000,5.469824,7.624853,13.0,44.0,13.0,...,71.0,22.0,1007.7,1007.1,8.0,7.0,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.1,0.000000,5.469824,7.624853,14.0,44.0,6.0,...,44.0,25.0,1010.6,1007.8,7.0,7.0,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.7,0.000000,5.469824,7.624853,15.0,46.0,13.0,...,38.0,30.0,1007.6,1008.7,7.0,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.0,0.000000,5.469824,7.624853,4.0,24.0,9.0,...,45.0,16.0,1017.6,1012.8,7.0,7.0,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.3,1.000000,5.469824,7.624853,13.0,41.0,1.0,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,Yes
5,2008-12-06,Albury,14.6,29.7,0.200000,5.469824,7.624853,14.0,56.0,13.0,...,55.0,23.0,1009.2,1005.4,7.0,7.0,20.6,28.9,No,No
6,2008-12-07,Albury,14.3,25.0,0.000000,5.469824,7.624853,13.0,50.0,12.0,...,49.0,19.0,1009.6,1008.2,1.0,7.0,18.1,24.6,No,No
7,2008-12-08,Albury,7.7,26.7,0.000000,5.469824,7.624853,13.0,35.0,10.0,...,48.0,19.0,1013.4,1010.1,7.0,7.0,16.3,25.5,No,No
8,2008-12-09,Albury,9.7,31.9,0.000000,5.469824,7.624853,6.0,80.0,9.0,...,42.0,9.0,1008.9,1003.6,7.0,7.0,18.3,30.2,Yes,No
9,2008-12-10,Albury,13.1,30.1,1.400000,5.469824,7.624853,13.0,28.0,8.0,...,58.0,27.0,1007.0,1005.7,7.0,7.0,20.1,28.2,No,Yes


In [28]:
weather["Date"] = weather["Date"].apply(lambda x:int(x.split("-")[1]))
#apply是对dataframe上的某一列进行处理的一个函数
#lambda x匿名函数，请在dataframe上这一列中的每一行帮我执行冒号后的命令

In [29]:
weather.loc[:,"Date"].value_counts()

5     13055
3     13036
1     12921
6     12389
10    11804
7     11779
8     11762
11    11461
9     11345
4     11306
12    10810
2     10525
Name: Date, dtype: int64

In [30]:
#替换完毕后，我们需要修改列的名称
#rename是比较少用的，可以用来修改单个列名的函数
#我们通常都直接使用 df.columns = 某个列表 这样的形式来一次修改所有的列名
#但rename允许我们只修改某个单独的列
weather = weather.rename(columns={"Date":"Month"})
weather.head()

,Month,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,RainToday
0,12,Albury,13.4,22.9,0.6,5.469824,7.624853,13.0,44.0,13.0,...,71.0,22.0,1007.7,1007.1,8.0,7.0,16.9,21.8,No,No
1,12,Albury,7.4,25.1,0.0,5.469824,7.624853,14.0,44.0,6.0,...,44.0,25.0,1010.6,1007.8,7.0,7.0,17.2,24.3,No,No
2,12,Albury,12.9,25.7,0.0,5.469824,7.624853,15.0,46.0,13.0,...,38.0,30.0,1007.6,1008.7,7.0,2.0,21.0,23.2,No,No
3,12,Albury,9.2,28.0,0.0,5.469824,7.624853,4.0,24.0,9.0,...,45.0,16.0,1017.6,1012.8,7.0,7.0,18.1,26.5,No,No
4,12,Albury,17.5,32.3,1.0,5.469824,7.624853,13.0,41.0,1.0,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,Yes


In [31]:
# 处理地点

In [32]:
# 城市-经纬度，经纬度-气候，把两张表合成一张表
cityll = pd.read_csv(r"/Users/williamchen/Downloads/IDS/rain/cityll.csv",index_col=0)
city_climate = pd.read_csv(r"/Users/williamchen/Downloads/IDS/rain/Cityclimate.csv")

In [33]:
cityll.head() #每个城市对应的经纬度，这些城市是澳大利亚统计局做的那张地图上的城市

,City,Latitude,Longitude,Latitudedir,Longitudedir
0,Adelaide,34.9285°,138.6007°,"S,",E
1,Albany,35.0275°,117.8840°,"S,",E
2,Albury,36.0737°,146.9135°,"S,",E
3,Wodonga,36.1241°,146.8818°,"S,",E
4,AliceSprings,23.6980°,133.8807°,"S,",E


In [34]:
city_climate.head() #澳大利亚统计局做的每个城市对应的气候

,City,Climate
0,Adelaide,Warm temperate
1,Albany,Mild temperate
2,Albury,"Hot dry summer, cool winter"
3,Wodonga,"Hot dry summer, cool winter"
4,AliceSprings,"Hot dry summer, warm winter"


In [35]:
#去掉度数符号
cityll["Latitudenum"] = cityll["Latitude"].apply(lambda x:float(x[:-1]))
cityll["Longitudenum"] = cityll["Longitude"].apply(lambda x:float(x[:-1]))

In [36]:
#观察一下所有的经纬度方向都是一致的，全部是南纬，东经，因为澳大利亚在南半球，东半球
#所以经纬度的方向我们可以舍弃了
citylld = cityll.iloc[:,[0,5,6]]
citylld

,City,Latitudenum,Longitudenum
0,Adelaide,34.9285,138.6007
1,Albany,35.0275,117.8840
2,Albury,36.0737,146.9135
3,Wodonga,36.1241,146.8818
4,AliceSprings,23.6980,133.8807
...,...,...,...
95,Wollongong,34.4278,150.8931
96,Wyndham,15.4825,128.1228
97,Yalgoo,28.3445,116.6851
98,Yulara,25.2335,130.9849


In [37]:
#将city_climate中的气候添加到我们的citylld中
citylld["climate"] = city_climate.iloc[:,-1]
citylld.head()
# 新表citylld

/Users/williamchen/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,City,Latitudenum,Longitudenum,climate
0,Adelaide,34.9285,138.6007,Warm temperate
1,Albany,35.0275,117.8840,Mild temperate
2,Albury,36.0737,146.9135,"Hot dry summer, cool winter"
3,Wodonga,36.1241,146.8818,"Hot dry summer, cool winter"
4,AliceSprings,23.6980,133.8807,"Hot dry summer, warm winter"


In [38]:
# 数据里的城市
samplecity = pd.read_csv(r"/Users/williamchen/Downloads/IDS/rain/samplecity.csv",index_col=0)

In [39]:
samplecity.head()

,City,Latitude,Longitude,Latitudedir,Longitudedir
0,Canberra,35.2809°,149.1300°,"S,",E
1,Sydney,33.8688°,151.2093°,"S,",E
2,Perth,31.9505°,115.8605°,"S,",E
3,Darwin,12.4634°,130.8456°,"S,",E
4,Hobart,42.8821°,147.3272°,"S,",E


In [40]:
#我们对samplecity也执行同样的处理：去掉经纬度中度数的符号，并且舍弃我们的经纬度的方向
samplecity["Latitudenum"] = samplecity["Latitude"].apply(lambda x:float(x[:-1]))
samplecity["Longitudenum"] = samplecity["Longitude"].apply(lambda x:float(x[:-1]))
samplecityd = samplecity.iloc[:,[0,5,6]]
samplecityd.head()

,City,Latitudenum,Longitudenum
0,Canberra,35.2809,149.1300
1,Sydney,33.8688,151.2093
2,Perth,31.9505,115.8605
3,Darwin,12.4634,130.8456
4,Hobart,42.8821,147.3272


In [41]:
# 好了，我们现在有了澳大利亚主要城市的经纬度和对应的气候，也有了我们的样本的地点所对应的经纬度，
# 接下来我们要开始计算我们样本上的地点到每个澳大利亚主要城市的距离，
# 而离我们的样本地点最近的那个澳大利亚主要城市的气候，就是我们样本点的气候。

In [42]:
#首先使用radians将角度转换成弧度
from math import radians, sin, cos, acos
citylld.loc[:,"slat"] = citylld.iloc[:,1].apply(lambda x : radians(x))
citylld.loc[:,"slon"] = citylld.iloc[:,2].apply(lambda x : radians(x))
samplecityd.loc[:,"elat"] = samplecityd.iloc[:,1].apply(lambda x : radians(x))
samplecityd.loc[:,"elon"] = samplecityd.iloc[:,2].apply(lambda x : radians(x))

/Users/williamchen/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/williamchen/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [43]:
import sys
for i in range(samplecityd.shape[0]):
    abcd = citylld.loc[:,"slat"]
    efdg = citylld.loc[:,"slon"]
    asdas = samplecityd.loc[i,"elat"]
    asdas = samplecityd.loc[i,"elon"]
    fasfa = 6371.01 * np.arccos(np.sin(slat)*np.sin(elat) + 
                          np.cos(slat)*np.cos(elat)*np.cos(slon.values - elon))
    city_index = np.argsort(dist)[0]
    
    samplecityd.loc[i,"closest_city"] = citylld.loc[city_index,"City"]
    samplecityd.loc[i,"climate"] = citylld.loc[city_index,"climate"]

/Users/williamchen/opt/anaconda3/lib/python3.7/site-packages/pandas/core/series.py:679: RuntimeWarning: invalid value encountered in arccos
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [44]:
#查看最后的结果，需要检查城市匹配是否基本正确
samplecityd.head(5)

,City,Latitudenum,Longitudenum,elat,elon,closest_city,climate
0,Canberra,35.2809,149.1300,0.615768,2.602810,Canberra,Cool temperate
1,Sydney,33.8688,151.2093,0.591122,2.639100,Sydney,Warm temperate
2,Perth,31.9505,115.8605,0.557641,2.022147,Perth,Warm temperate
3,Darwin,12.4634,130.8456,0.217527,2.283687,Darwin,"High humidity summer, warm winter"
4,Hobart,42.8821,147.3272,0.748434,2.571345,Hobart,Cool temperate


In [45]:
#查看气候的分布
samplecityd["climate"].value_counts()

Warm temperate                       15
Mild temperate                       10
Cool temperate                        9
Hot dry summer, cool winter           6
High humidity summer, warm winter     4
Hot dry summer, warm winter           3
Warm humid summer, mild winter        2
Name: climate, dtype: int64

In [46]:
#确认无误后，取出样本城市所对应的气候，并保存
locafinal = samplecityd.iloc[:,[0,-1]]
locafinal.head()

,City,climate
0,Canberra,Cool temperate
1,Sydney,Warm temperate
2,Perth,Warm temperate
3,Darwin,"High humidity summer, warm winter"
4,Hobart,Cool temperate


In [47]:
locafinal.columns = ["Location","Climate"]
#在这里设定locafinal的索引为地点，是为了之后进行map的匹配
locafinal = locafinal.set_index(keys="Location")
locafinal.head()

,Climate
Location,
Canberra,Cool temperate
Sydney,Warm temperate
Perth,Warm temperate
Darwin,"High humidity summer, warm winter"
Hobart,Cool temperate


In [48]:
weather.head()

,Month,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,RainToday
0,12,Albury,13.4,22.9,0.6,5.469824,7.624853,13.0,44.0,13.0,...,71.0,22.0,1007.7,1007.1,8.0,7.0,16.9,21.8,No,No
1,12,Albury,7.4,25.1,0.0,5.469824,7.624853,14.0,44.0,6.0,...,44.0,25.0,1010.6,1007.8,7.0,7.0,17.2,24.3,No,No
2,12,Albury,12.9,25.7,0.0,5.469824,7.624853,15.0,46.0,13.0,...,38.0,30.0,1007.6,1008.7,7.0,2.0,21.0,23.2,No,No
3,12,Albury,9.2,28.0,0.0,5.469824,7.624853,4.0,24.0,9.0,...,45.0,16.0,1017.6,1012.8,7.0,7.0,18.1,26.5,No,No
4,12,Albury,17.5,32.3,1.0,5.469824,7.624853,13.0,41.0,1.0,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,Yes


In [49]:
#将location中的内容替换，并且确保匹配进入的气候字符串中不含有逗号，气候两边不含有空格
#我们使用re这个模块来消除逗号
#re.sub(希望替换的值，希望被替换成的值，要操作的字符串) #去掉逗号
#x.strip()是去掉空格的函数
#把location替换成气候的是我们的map的映射
import re
#气象站的名字替换成了对应的城市对应的气候
weather["Location"] = weather["Location"].map(locafinal.iloc[:,0])
weather.head()

,Month,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,RainToday
0,12,"Hot dry summer, cool winter",13.4,22.9,0.6,5.469824,7.624853,13.0,44.0,13.0,...,71.0,22.0,1007.7,1007.1,8.0,7.0,16.9,21.8,No,No
1,12,"Hot dry summer, cool winter",7.4,25.1,0.0,5.469824,7.624853,14.0,44.0,6.0,...,44.0,25.0,1010.6,1007.8,7.0,7.0,17.2,24.3,No,No
2,12,"Hot dry summer, cool winter",12.9,25.7,0.0,5.469824,7.624853,15.0,46.0,13.0,...,38.0,30.0,1007.6,1008.7,7.0,2.0,21.0,23.2,No,No
3,12,"Hot dry summer, cool winter",9.2,28.0,0.0,5.469824,7.624853,4.0,24.0,9.0,...,45.0,16.0,1017.6,1012.8,7.0,7.0,18.1,26.5,No,No
4,12,"Hot dry summer, cool winter",17.5,32.3,1.0,5.469824,7.624853,13.0,41.0,1.0,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,Yes


In [50]:
#城市的气候中所含的逗号和空格都去掉
weather["Location"] = weather["Location"].apply(lambda x:re.sub(",","",x.strip()))

In [51]:
#修改特征内容之后，我们使用新列名“Climate”来替换之前的列名“Location”
#注意这个命令一旦执行之后，就再没有列"Location"了，使用索引时要特别注意
weather = weather.rename(columns={"Location":"Climate"})

In [52]:
weather.head()

,Month,Climate,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,RainToday
0,12,Hot dry summer cool winter,13.4,22.9,0.6,5.469824,7.624853,13.0,44.0,13.0,...,71.0,22.0,1007.7,1007.1,8.0,7.0,16.9,21.8,No,No
1,12,Hot dry summer cool winter,7.4,25.1,0.0,5.469824,7.624853,14.0,44.0,6.0,...,44.0,25.0,1010.6,1007.8,7.0,7.0,17.2,24.3,No,No
2,12,Hot dry summer cool winter,12.9,25.7,0.0,5.469824,7.624853,15.0,46.0,13.0,...,38.0,30.0,1007.6,1008.7,7.0,2.0,21.0,23.2,No,No
3,12,Hot dry summer cool winter,9.2,28.0,0.0,5.469824,7.624853,4.0,24.0,9.0,...,45.0,16.0,1017.6,1012.8,7.0,7.0,18.1,26.5,No,No
4,12,Hot dry summer cool winter,17.5,32.3,1.0,5.469824,7.624853,13.0,41.0,1.0,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,Yes


In [53]:
#查看缺失值的缺失情况
weather.isnull().mean()

Month            0.0
Climate          0.0
MinTemp          0.0
MaxTemp          0.0
Rainfall         0.0
Evaporation      0.0
Sunshine         0.0
WindGustDir      0.0
WindGustSpeed    0.0
WindDir9am       0.0
WindDir3pm       0.0
WindSpeed9am     0.0
WindSpeed3pm     0.0
Humidity9am      0.0
Humidity3pm      0.0
Pressure9am      0.0
Pressure3pm      0.0
Cloud9am         0.0
Cloud3pm         0.0
Temp9am          0.0
Temp3pm          0.0
RainTomorrow     0.0
RainToday        0.0
dtype: float64

In [54]:
weather.shape

(142193, 23)

In [55]:
weather.head(50)

,Month,Climate,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,RainToday
0,12,Hot dry summer cool winter,13.4,22.9,0.600000,5.469824,7.624853,13.0,44.0,13.0,...,71.0,22.0,1007.7,1007.1,8.0,7.0,16.9,21.8,No,No
1,12,Hot dry summer cool winter,7.4,25.1,0.000000,5.469824,7.624853,14.0,44.0,6.0,...,44.0,25.0,1010.6,1007.8,7.0,7.0,17.2,24.3,No,No
2,12,Hot dry summer cool winter,12.9,25.7,0.000000,5.469824,7.624853,15.0,46.0,13.0,...,38.0,30.0,1007.6,1008.7,7.0,2.0,21.0,23.2,No,No
3,12,Hot dry summer cool winter,9.2,28.0,0.000000,5.469824,7.624853,4.0,24.0,9.0,...,45.0,16.0,1017.6,1012.8,7.0,7.0,18.1,26.5,No,No
4,12,Hot dry summer cool winter,17.5,32.3,1.000000,5.469824,7.624853,13.0,41.0,1.0,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,Yes
5,12,Hot dry summer cool winter,14.6,29.7,0.200000,5.469824,7.624853,14.0,56.0,13.0,...,55.0,23.0,1009.2,1005.4,7.0,7.0,20.6,28.9,No,No
6,12,Hot dry summer cool winter,14.3,25.0,0.000000,5.469824,7.624853,13.0,50.0,12.0,...,49.0,19.0,1009.6,1008.2,1.0,7.0,18.1,24.6,No,No
7,12,Hot dry summer cool winter,7.7,26.7,0.000000,5.469824,7.624853,13.0,35.0,10.0,...,48.0,19.0,1013.4,1010.1,7.0,7.0,16.3,25.5,No,No
8,12,Hot dry summer cool winter,9.7,31.9,0.000000,5.469824,7.624853,6.0,80.0,9.0,...,42.0,9.0,1008.9,1003.6,7.0,7.0,18.3,30.2,Yes,No
9,12,Hot dry summer cool winter,13.1,30.1,1.400000,5.469824,7.624853,13.0,28.0,8.0,...,58.0,27.0,1007.0,1005.7,7.0,7.0,20.1,28.2,No,Yes


In [56]:
weather.describe([0.01,0.05,0.1,0.25,0.5,0.75,0.9,0.99]).T

,count,mean,std,min,1%,5%,10%,25%,50%,75%,90%,99%,max
Month,142193.0,6.402544,3.426506,1.0,1.0,1.0,2.0,3.000000,6.000000,9.000000,11.0,12.0,12.0
MinTemp,142193.0,12.186400,6.388924,-8.5,-1.8,1.8,4.0,7.600000,12.000000,16.800000,20.8,25.8,33.9
MaxTemp,142193.0,23.226784,7.109554,-4.8,9.1,12.8,14.5,17.900000,22.700000,28.200000,32.9,40.1,48.1
Rainfall,142193.0,2.349974,8.423217,0.0,0.0,0.0,0.0,0.000000,0.000000,0.800000,6.0,37.2,371.0
Evaporation,142193.0,5.469824,3.168114,0.0,0.4,1.2,2.0,4.000000,5.469824,5.469824,8.2,15.8,145.0
Sunshine,142193.0,7.624853,2.734927,0.0,0.0,1.4,3.7,7.624853,7.624853,8.700000,11.1,13.2,14.5
WindGustDir,142193.0,8.059039,4.694267,0.0,0.0,0.0,1.0,4.000000,9.000000,13.000000,14.0,15.0,15.0
WindGustSpeed,142193.0,39.984292,13.138385,6.0,15.0,20.0,24.0,31.000000,39.000000,46.000000,57.0,80.0,135.0
WindDir9am,142193.0,7.000316,4.514678,0.0,0.0,0.0,1.0,3.000000,7.000000,11.000000,13.0,15.0,15.0
WindDir3pm,142193.0,7.788485,4.546322,0.0,0.0,0.0,1.0,4.000000,8.000000,12.000000,14.0,15.0,15.0


In [58]:
# weather.to_csv("/Users/williamchen/Downloads/IDS/rain/weather-14w.csv")

In [16]:
import pandas as pd
import numpy as np

In [17]:
weather = pd.read_csv(r"/Users/williamchen/Downloads/IDS/rain/weather-14w-old.csv",index_col=0)

In [18]:
category = ['Climate',
 'WindGustDir',
 'WindDir9am',
 'WindDir3pm',
 'Cloud9am',
 'Cloud3pm']

In [19]:
#将所有的分类型变量编码为数字，一个类别是一个数字
from sklearn.preprocessing import OrdinalEncoder #只允许二维以上的数据进行输入
oe = OrdinalEncoder()

In [20]:
oe = oe.fit(weather.loc[:,category])

In [21]:
#用训练集的编码结果来编码训练和测试特征矩阵
#在这里如果测试特征矩阵报错，就说明测试集中出现了训练集中从未见过的类别
weather.loc[:,category] = oe.transform(weather.loc[:,category])

In [22]:
weather.loc[:,category].head()

,Climate,WindGustDir,WindDir9am,WindDir3pm,Cloud9am,Cloud3pm
0,2.0,13.0,13.0,14.0,8.0,7.0
1,2.0,14.0,6.0,15.0,7.0,7.0
2,2.0,15.0,13.0,15.0,7.0,2.0
3,2.0,4.0,9.0,0.0,7.0,7.0
4,2.0,13.0,1.0,7.0,7.0,8.0


In [23]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Month          142193 non-null  int64  
 1   Climate        142193 non-null  float64
 2   MinTemp        142193 non-null  float64
 3   MaxTemp        142193 non-null  float64
 4   Rainfall       142193 non-null  float64
 5   Evaporation    142193 non-null  float64
 6   Sunshine       142193 non-null  float64
 7   WindGustDir    142193 non-null  float64
 8   WindGustSpeed  142193 non-null  float64
 9   WindDir9am     142193 non-null  float64
 10  WindDir3pm     142193 non-null  float64
 11  WindSpeed9am   142193 non-null  float64
 12  WindSpeed3pm   142193 non-null  float64
 13  Humidity9am    142193 non-null  float64
 14  Humidity3pm    142193 non-null  float64
 15  Pressure9am    142193 non-null  float64
 16  Pressure3pm    142193 non-null  float64
 17  Cloud9am       142193 non-nul

In [24]:
pd.set_option('display.max_columns', None)
weather.head()

,Month,Climate,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,RainToday
0,12,2.0,13.4,22.9,0.6,5.469824,7.624853,13.0,44.0,13.0,14.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,7.0,16.9,21.8,No,No
1,12,2.0,7.4,25.1,0.0,5.469824,7.624853,14.0,44.0,6.0,15.0,4.0,22.0,44.0,25.0,1010.6,1007.8,7.0,7.0,17.2,24.3,No,No
2,12,2.0,12.9,25.7,0.0,5.469824,7.624853,15.0,46.0,13.0,15.0,19.0,26.0,38.0,30.0,1007.6,1008.7,7.0,2.0,21.0,23.2,No,No
3,12,2.0,9.2,28.0,0.0,5.469824,7.624853,4.0,24.0,9.0,0.0,11.0,9.0,45.0,16.0,1017.6,1012.8,7.0,7.0,18.1,26.5,No,No
4,12,2.0,17.5,32.3,1.0,5.469824,7.624853,13.0,41.0,1.0,7.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,Yes


In [25]:
# Target content and RainToday changed to 1-0.
# Column RainToday was Yes when the values in column Rainfall were greater than 1, and No when it was less than 1. Accordingly, a filling process will be made
weather.RainTomorrow = [0 if each=='No' else 1 for each in weather.RainTomorrow]
weather.RainToday = [0 if each=='No' else 1 for each in weather.RainToday]

In [26]:
pd.set_option('display.max_columns', None)
weather.head(50)

,Month,Climate,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainTomorrow,RainToday
0,12,2.0,13.4,22.9,0.600000,5.469824,7.624853,13.0,44.0,13.0,14.0,20.000000,24.0,71.0,22.0,1007.7,1007.1,8.0,7.0,16.9,21.8,0,0
1,12,2.0,7.4,25.1,0.000000,5.469824,7.624853,14.0,44.0,6.0,15.0,4.000000,22.0,44.0,25.0,1010.6,1007.8,7.0,7.0,17.2,24.3,0,0
2,12,2.0,12.9,25.7,0.000000,5.469824,7.624853,15.0,46.0,13.0,15.0,19.000000,26.0,38.0,30.0,1007.6,1008.7,7.0,2.0,21.0,23.2,0,0
3,12,2.0,9.2,28.0,0.000000,5.469824,7.624853,4.0,24.0,9.0,0.0,11.000000,9.0,45.0,16.0,1017.6,1012.8,7.0,7.0,18.1,26.5,0,0
4,12,2.0,17.5,32.3,1.000000,5.469824,7.624853,13.0,41.0,1.0,7.0,7.000000,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,1
5,12,2.0,14.6,29.7,0.200000,5.469824,7.624853,14.0,56.0,13.0,13.0,19.000000,24.0,55.0,23.0,1009.2,1005.4,7.0,7.0,20.6,28.9,0,0
6,12,2.0,14.3,25.0,0.000000,5.469824,7.624853,13.0,50.0,12.0,13.0,20.000000,24.0,49.0,19.0,1009.6,1008.2,1.0,7.0,18.1,24.6,0,0
7,12,2.0,7.7,26.7,0.000000,5.469824,7.624853,13.0,35.0,10.0,13.0,6.000000,17.0,48.0,19.0,1013.4,1010.1,7.0,7.0,16.3,25.5,0,0
8,12,2.0,9.7,31.9,0.000000,5.469824,7.624853,6.0,80.0,9.0,7.0,7.000000,28.0,42.0,9.0,1008.9,1003.6,7.0,7.0,18.3,30.2,1,0
9,12,2.0,13.1,30.1,1.400000,5.469824,7.624853,13.0,28.0,8.0,10.0,15.000000,11.0,58.0,27.0,1007.0,1005.7,7.0,7.0,20.1,28.2,0,1


In [27]:
weather.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142193 entries, 0 to 142192
Data columns (total 23 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   Month          142193 non-null  int64  
 1   Climate        142193 non-null  float64
 2   MinTemp        142193 non-null  float64
 3   MaxTemp        142193 non-null  float64
 4   Rainfall       142193 non-null  float64
 5   Evaporation    142193 non-null  float64
 6   Sunshine       142193 non-null  float64
 7   WindGustDir    142193 non-null  float64
 8   WindGustSpeed  142193 non-null  float64
 9   WindDir9am     142193 non-null  float64
 10  WindDir3pm     142193 non-null  float64
 11  WindSpeed9am   142193 non-null  float64
 12  WindSpeed3pm   142193 non-null  float64
 13  Humidity9am    142193 non-null  float64
 14  Humidity3pm    142193 non-null  float64
 15  Pressure9am    142193 non-null  float64
 16  Pressure3pm    142193 non-null  float64
 17  Cloud9am       142193 non-nul

In [28]:
weather.to_csv("/Users/williamchen/Downloads/IDS/rain/weather-14w.csv")

In [ ]:
# 完结